# Projet 4

In [12]:
import requests
from bs4 import BeautifulSoup
import re
import json
import xml.etree.ElementTree as ET
from multiprocessing import Pool
import sub_func

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

ModuleNotFoundError: No module named 'sub_func'

Ecrire une fonction `get_prices_from_url()` qui extrait des informations à partir des 2 pages ci-dessous.

Exemple `URL_PAGE2` doit retourner :

<pre>{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
  'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
  'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
</pre>

In [2]:
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"

def get_prices_from_url(url):
    r =requests.get(url)
    soup = BeautifulSoup(r.content)
    parts = soup.find_all('div',{'class' : re.compile("pricing-table pricing-table-")})
    prices = {}
    for part in parts:
        contract_type = part.findChild("h2").text
        details = {
            'price'     : '$'+re.findall(r'\d+', part.findChild("span").text)[0],
            'storage'   :  part.findChild("ul").text.split('\n')[4].replace(" file storage",""),
            'databases' : int(re.findall(r'\d+',part.findChild("ul").text.split('\n')[5])[0])
        }
        print(details)
        prices[contract_type] = details
    return prices

In [3]:
get_prices_from_url("https://kim.fspot.org/cours/page2.html")


{'price': '$5', 'storage': '1GB', 'databases': 1}
{'price': '$25', 'storage': '10GB', 'databases': 5}
{'price': '$45', 'storage': '100GB', 'databases': 25}


{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}

Ecrire une fonction qui extrait des informations sur une bière de beowulf.

Exemple d'URL: https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33 

La fonction doit retourner :
<pre>
{'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33}
</pre>

In [4]:
def extract_beer_infos(url):
    r =requests.get(url)
    soup = BeautifulSoup(r.content)
    infos = {
        'name': soup.find('title').text.split("|")[0].strip(),
        'note': int(soup.find_all('div',{'class' : "stars"})[0]["data-percent"]),
        'price': float(soup.find_all('div',{'class' : "price-container"})[0].find("span").text.replace("€","").replace(",",".")),
        'volume': int(soup.find_all('div',{'class' : "product-subtext"})[0].text.replace("\n","").split("|")[2].replace("cl","").strip())
    }
    return infos

In [5]:
extract_beer_infos('https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33')

{'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33}

Cette URL retourne un JSON avec une liste de bières :

In [6]:
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"

Ecrire une fonction qui prend l'argument cet URL retourne les informations sur une liste de bière via l'API de beowulf.

Cette fonction doit retourner la liste des informations obtenues par la fonction `extract_beer_infos()` définie ci-dessus.

Chercher comment optimiser cette fonction en utilisant multiprocessing.Pool pour paralléliser les accès web.

Exemple de retour :

<pre>[{'name': 'Gallia East IPA', 'note': 80, 'price': 42.99, 'volume': 33},
    {'name': 'La Lager Sans Gluten de Vézelay', 'note': 60, 'price': 38.99, 'volume': 25},
    {'name': 'Brasserie De Sutter Brin de Folie', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'La Cristal IPA du Mont Blanc', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'La Parisienne Le Titi Parisien', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'Gallia Session IPA', 'note': 70, 'price': 42.99, 'volume': 33},
    {'name': 'Ninkasi Brut IPA', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Pietra', 'note': 60, 'price': 38.99, 'volume': 33},
    {'name': 'Desperados', 'note': 60, 'price': 35.99, 'volume': 33},
    {'name': 'Gallia West IPA', 'note': 70, 'price': 42.99, 'volume': 33}]
</pre>

In [7]:
def extract_beer_list_infos(url):
    r =requests.get(URL_BEERLIST_FRANCE)
    soup = BeautifulSoup(r.content)
    parsed =json.loads(soup.find_all("p")[0].text)
    items = parsed['items']
    beer_pages = []
    for i in range(0, len(items)):
        beer_pages.append('https://www.beerwulf.com'+items[i]['contentReference'])

    beers = []
    # Sequential version (slow):
    for beer in beer_pages:
       beers.append(extract_beer_infos(beer))
    # Parallel version (faster):
    # with Pool(1) as p:
    #     beers = p.map(extract_beer_infos, beer_pages)
    return beers

In [8]:
extract_beer_list_infos(URL_BEERLIST_FRANCE)


[{'name': 'Gallia East IPA', 'note': 80, 'price': 42.99, 'volume': 33},
 {'name': 'La Lager Sans Gluten de Vézelay',
  'note': 60,
  'price': 38.99,
  'volume': 25},
 {'name': 'Brasserie De Sutter Brin de Folie',
  'note': 70,
  'price': 44.99,
  'volume': 33},
 {'name': 'La Cristal IPA du Mont Blanc',
  'note': 70,
  'price': 44.99,
  'volume': 33},
 {'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33},
 {'name': 'La Parisienne Le Titi Parisien',
  'note': 70,
  'price': 38.99,
  'volume': 33},
 {'name': 'Gallia Session IPA', 'note': 70, 'price': 42.99, 'volume': 33},
 {'name': 'Ninkasi Brut IPA', 'note': 70, 'price': 44.99, 'volume': 33},
 {'name': 'Pietra', 'note': 60, 'price': 38.99, 'volume': 33},
 {'name': 'Desperados', 'note': 60, 'price': 35.99, 'volume': 33},
 {'name': 'Gallia West IPA', 'note': 70, 'price': 42.99, 'volume': 33}]

In [9]:
r =requests.get(URL_BEERLIST_FRANCE)
soup = BeautifulSoup(r.content)
parsed =json.loads(soup.find_all("p")[0].text)
items = parsed['items']
cont = items[3]['contentReference']
r =requests.get('https://www.beerwulf.com'+cont)
extract_beer_infos('https://www.beerwulf.com'+cont)


{'name': 'La Cristal IPA du Mont Blanc',
 'note': 70,
 'price': 44.99,
 'volume': 33}

In [10]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_01_get_prices_from_url_page2(self):
        prices = get_prices_from_url(URL_PAGE2)
        # We should have found 3 products:
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 3)
        self.assertIn('Personal', prices)
        self.assertIn('Small Business', prices)
        self.assertIn('Enterprise', prices)
        
        personal = prices['Personal']
        self.assertIn('price', personal)
        self.assertIn('storage', personal)
        self.assertIn('databases', personal)
        self.assertEqual(personal['price'], '$5')
        self.assertEqual(personal['storage'], '1GB')
        self.assertEqual(personal['databases'], 1)
        
    def test_02_get_prices_from_url_page3(self):
        prices = get_prices_from_url(URL_PAGE3)
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 4)
        self.assertEqual(
            prices['Privilege'],
            {'databases': 100, 'price': '$99', 'storage': '1TB'}
        )
    
    def test_03_extract_beer_list_infos(self):
        infos = extract_beer_list_infos(URL_BEERLIST_FRANCE)
        # We should have 11 French beers:
        self.assertIsInstance(infos, list)
        self.assertEqual(len(infos), 11)
        # All of them are 25cl or 33cl:
        for beer in infos:
            self.assertIn(beer['volume'], [25, 33])

            
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [11]:
if __name__ == '__main__':
    run_tests()

test_01_get_prices_from_url_page2 (__main__.Lesson4Tests) ... ok
test_02_get_prices_from_url_page3 (__main__.Lesson4Tests) ... ok
test_03_extract_beer_list_infos (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 3.130s

OK


{'price': '$5', 'storage': '1GB', 'databases': 1}
{'price': '$25', 'storage': '10GB', 'databases': 5}
{'price': '$45', 'storage': '100GB', 'databases': 25}
{'price': '$5', 'storage': '1GB', 'databases': 1}
{'price': '$25', 'storage': '10GB', 'databases': 5}
{'price': '$45', 'storage': '100GB', 'databases': 25}
{'price': '$99', 'storage': '1TB', 'databases': 100}
